In [1]:
import sys
# print(sys.path)
sys.path.append("/home/abdullahm/jaleel/Review_analysis")

['/home/abdullahm/jaleel/Review_analysis/notebooks', '', '/home/abdullahm/miniconda3/envs/aaai2/lib/python3.11/site-packages', '/home/abdullahm/miniconda3/envs/jaleel_btp/lib/python313.zip', '/home/abdullahm/miniconda3/envs/jaleel_btp/lib/python3.13', '/home/abdullahm/miniconda3/envs/jaleel_btp/lib/python3.13/lib-dynload', '/home/abdullahm/miniconda3/envs/jaleel_btp/lib/python3.13/site-packages']


In [51]:
from src.util.config import load_config

cfg = load_config()


### convert Raw data to json and save it


In [5]:
import pickle as pkl
import json

def pkl_to_json(src_path, target_path):
    # load pickle (binary mode)
    with open(src_path, 'rb') as f:  
        data = pkl.load(f)
    
    # save JSON (text mode)
    with open(target_path, 'w') as f:
        json.dump(data, f, indent=2, default=str)  
        # default=str converts unsupported types to string


In [8]:
pkl_to_json(f"{cfg["paths"]['data_dir']}/raw/test_all.pkl", f"{cfg["paths"]["data_dir"]}/test.json")

### split question and answer in dataset


In [23]:
import json
import re

# Paths to your JSON files
train_json_path = cfg["paths"]['data_dir']+"/train.json"
test_json_path  = cfg["paths"]['data_dir']+"/test.json"

output_train_json = cfg["paths"]['data_dir']+"/train_split.json"
output_test_json  = cfg["paths"]['data_dir']+"/test_split.json"

# List of questions
questions = [
    "According to you , what is going well in this course",
    "Is there anything you wish to be changed",
    "Share your suggestions on how the experience could have improved",
    "What helped you learn in this course",
    "What changes would improve your learning",
    "Comments on labs (if any) with respect to the course and your learning",
    "What suggestions do you have for another student considering taking this course",
    "Comments on teaching (if any) assistant(s) with respect to the course and your learning",
    "Comments on tutorials (if any) with respect to the course and your learning",
    "Any comment you want to make on TAs work and labs",
    "According to you, what is going well in the course",
    "According to you what is going well in the course",
    "What changes would improve learning",
    "What helped you learned in this course",
    "According to you, what is going well in this course? Name atleast one."
]

def split_sentence_to_qa(data):
    new_data = []
    for item in data:
        sentence = item.get("sentence", "")
        matched_question = None
        for q in questions:
            # Case-insensitive match; remove extra spaces
            if sentence.lower().startswith(q.lower()):
                matched_question = q
                break
        if matched_question:
            # Extract answer
            answer = sentence[len(matched_question):].strip(" :,.-")
            item["question"] = matched_question
            item["answer"] = answer
        else:
            # If no match, leave question empty, answer = sentence
            print("NO MATCH", item.get("sentence") )
            # item["question"] = ""
            # item["answer"] = sentence
        new_data.append(item)
    return new_data

def process_json(input_path, output_path):
    # Load JSON
    with open(input_path, "r") as f:
        data = json.load(f)
    
    # Split sentences
    new_data = split_sentence_to_qa(data)
    
    # Save new JSON
    with open(output_path, "w") as f:
        json.dump(new_data, f, indent=4)
    
    print(f"Saved {output_path}")

# Process train and test JSON
process_json(train_json_path, output_train_json)
process_json(test_json_path, output_test_json)


Saved /home/abdullahm/jaleel/Review_analysis/data/train_split.json
Saved /home/abdullahm/jaleel/Review_analysis/data/test_split.json


### count of leaves

In [42]:
import json
import pandas as pd

# -------------------------------
# Step 1: Load dataset
# -------------------------------
with open("/home/abdullahm/jaleel/Review_analysis/data/train_pruned.json", "r") as f:
    train_data = json.load(f)

with open("/home/abdullahm/jaleel/Review_analysis/data/test_pruned.json", "r") as f:
    test_data = json.load(f)

all_data = test_data
df = pd.DataFrame(all_data)

# -------------------------------
# Step 2: Load category tree
# -------------------------------
with open("/home/abdullahm/jaleel/Review_analysis/data/new_category_tree.json", "r") as f:
    category_tree = json.load(f)

# -------------------------------
# Step 3: Function to get all category paths
# -------------------------------
def get_paths(node, prefix=""):
    """
    Recursively traverse the tree node to generate all category paths.
    node can be a dict with 'text' and 'children', or a string (leaf)
    """
    paths = []
    
    if isinstance(node, str):
        # Leaf node
        paths.append(prefix + node)
    elif isinstance(node, dict):
        # Node with 'text' and 'children'
        current_prefix = prefix + node["text"]
        children = node.get("children", [])
        if not children:
            paths.append(current_prefix)  # no children → leaf
        else:
            for child in children:
                paths.extend(get_paths(child, current_prefix + " > "))
    elif isinstance(node, list):
        for item in node:
            paths.extend(get_paths(item, prefix))
    return paths

# Get all possible category paths
all_category_paths = get_paths(category_tree)

# -------------------------------
# Step 4: Count occurrences for each path
# -------------------------------
counts = {}
for path in all_category_paths:
    counts[path] = df[df["categoryPath"] == path].shape[0]

# Convert to DataFrame for nicer display
counts_df = pd.DataFrame(list(counts.items()), columns=["Category Path", "Count"])
counts_df = counts_df.sort_values("Count", ascending=False).reset_index(drop=True)

# Show results
counts_df.head(20)  # display top 20

# Optional: save to CSV
counts_df.to_csv("/home/abdullahm/jaleel/Review_analysis/data/category_path_counts_full.csv", index=False)


### prune leaves with less than < 4 counts 

In [40]:
import json
from collections import Counter

# ---------- Function to get all leaf paths ----------
def get_paths(node, prefix=""):
    paths = []
    if isinstance(node, str):
        paths.append(prefix + node)
    elif isinstance(node, dict):
        current_prefix = prefix + node["text"]
        children = node.get("children", [])
        if not children:
            paths.append(current_prefix)
        else:
            for child in children:
                paths.extend(get_paths(child, current_prefix + " > "))
    elif isinstance(node, list):
        for item in node:
            paths.extend(get_paths(item, prefix))
    return paths

# ---------- Function to prune tree ----------
def prune_tree(node, leaf_counts, prefix=""):
    """
    Recursively prune children whose full leaf path occurs less than 4 times.
    """
    if isinstance(node, str):
        full_path = prefix + node
        return node if leaf_counts.get(full_path, 0) >= 4 else None
    elif isinstance(node, dict):
        current_path = prefix + node["text"]
        pruned_children = []
        for child in node.get("children", []):
            pruned_child = prune_tree(child, leaf_counts, current_path + " > ")
            if pruned_child is not None:
                pruned_children.append(pruned_child)
        if not pruned_children:
            # No children left, keep node as leaf only if its full path is frequent enough
            return node["text"] if leaf_counts.get(current_path, 0) >= 4 else None
        return {"text": node["text"], "children": pruned_children}
    elif isinstance(node, list):
        pruned_list = []
        for item in node:
            pruned_item = prune_tree(item, leaf_counts, prefix)
            if pruned_item is not None:
                pruned_list.append(pruned_item)
        return pruned_list if pruned_list else None

# ---------- Helper: get parent path of a leaf ----------
def get_leaf_to_parent_map(node, prefix=""):
    """
    Returns a dictionary mapping full leaf path -> parent full path
    """
    leaf_to_parent = {}
    if isinstance(node, str):
        leaf_to_parent[prefix + node] = prefix.rstrip(" > ")
    elif isinstance(node, dict):
        current_path = prefix + node["text"]
        for child in node.get("children", []):
            child_map = get_leaf_to_parent_map(child, current_path + " > ")
            leaf_to_parent.update(child_map)
    elif isinstance(node, list):
        for item in node:
            leaf_to_parent.update(get_leaf_to_parent_map(item, prefix))
    return leaf_to_parent

# ---------- Main Script ----------
if __name__ == "__main__":
    # 1. Load category tree
    with open("/home/abdullahm/jaleel/Review_analysis/data/category_tree.json") as f:
        category_tree = json.load(f)

    # 2. Load dataset (assume each item has a 'full_path' field)
    with open("/home/abdullahm/jaleel/Review_analysis/data/train_split.json") as f:
        dataset = json.load(f)
    with open("/home/abdullahm/jaleel/Review_analysis/data/test_split.json") as f:
        dataset_test = json.load(f)

    # 3. Count leaf occurrences by full path
    leaf_paths_in_data = [item["categoryPath"] for item in dataset]
    leaf_counts = Counter(leaf_paths_in_data)

    # 4. Prune the tree using full paths
    pruned_tree = prune_tree(category_tree, leaf_counts)

    # 5. Build leaf -> parent map from original tree
    leaf_to_parent = get_leaf_to_parent_map(category_tree)

    # 6. Update dataset: if leaf pruned, assign parent path
    for item in dataset:
        full_path = item["categoryPath"]
        if leaf_counts.get(full_path, 0) < 4:
            parent_path = leaf_to_parent.get(full_path, full_path)
            item["categoryPath"] = parent_path
            
    for item in dataset_test:
        full_path = item["categoryPath"]
        if leaf_counts.get(full_path, 0) < 4:
            parent_path = leaf_to_parent.get(full_path, full_path)
            item["categoryPath"] = parent_path

    # 7. Save pruned tree and updated dataset
    with open("/home/abdullahm/jaleel/Review_analysis/data/new_category_tree.json", "w") as f:
        json.dump(pruned_tree, f, indent=2)

    with open("/home/abdullahm/jaleel/Review_analysis/data/train_pruned.json", "w") as f:
        json.dump(dataset, f, indent=2)
    with open("/home/abdullahm/jaleel/Review_analysis/data/test_pruned.json", "w") as f:
        json.dump(dataset_test, f, indent=2)
        

    print("Pruning complete! New tree and dataset saved.")


Pruning complete! New tree and dataset saved.


### Add null to each internal node

In [45]:
import json

def add_null_children(node):
    # If it's a leaf (string), just return it
    if isinstance(node, str):
        return node
    
    # Recursively process children
    children = []
    for child in node.get("children", []):
        children.append(add_null_children(child))
    
    # Add a "null" child
    children.append("null")
    
    # Return updated node
    return {"text": node["text"], "children": children}

# Load tree
with open(cfg['paths']['data_dir']+"/new_category_tree.json", 'r') as f:
    tree = json.load(f)

# Wrap with root
new_tree = {"text": "root", "children": tree}

# Add null children
new_tree = add_null_children(new_tree)

# Save new tree
with open(cfg['paths']['data_dir'] + "/2new_category_tree.json", "w") as f:
    json.dump(new_tree, f, indent=2)


### Add null to categoryPaths ending with internal node


In [46]:
import json

def load_tree(tree_path):
    with open(tree_path, "r") as f:
        return json.load(f)

def build_internal_nodes_set(node, path=None, internal_nodes=None):
    """Collect full paths of all internal nodes (those with children)."""
    if internal_nodes is None:
        internal_nodes = set()
    if path is None:
        path = []

    if isinstance(node, dict):
        current_path = path + [node["text"]]
        children = node.get("children", [])
        if children:
            # Mark full path as internal
            internal_nodes.add(" > ".join(current_path))
            for child in children:
                build_internal_nodes_set(child, current_path, internal_nodes)

    return internal_nodes

def add_full_category_paths(data_path, tree_path, output_path):
    """Add 'full_category_path' to each datapoint safely (handles duplicate names)."""
    # Load data + tree
    with open(data_path, "r") as f:
        data = json.load(f)
    tree = load_tree(tree_path)

    # Collect full paths of internal nodes
    internal_nodes = build_internal_nodes_set(tree)

    for item in data:
        cat_path = item.get("categoryPath", "")
        path_parts = cat_path.split(" > ") if cat_path else []

        # Always prepend root
        full_path = ["root"] + path_parts

        # Build string representation
        full_path_str = " > ".join(full_path)

        # If this *exact full path* is internal → add "null"
        if full_path_str in internal_nodes:
            full_path.append("null")

        # Save back
        item["full_category_path"] = " > ".join(full_path)

    # Save updated dataset
    with open(output_path, "w") as f:
        json.dump(data, f, indent=2)

    print(f"✅ Updated data saved to {output_path}")


In [48]:
add_full_category_paths(
    data_path=cfg['paths']['data_dir'] + "/test_pruned.json",            # <-- your dataset file
    tree_path=cfg['paths']['data_dir'] + "/2new_category_tree.json", # <-- new tree
    output_path=cfg['paths']['data_dir'] + "/test_null_appended.json"
)


✅ Updated data saved to /home/abdullahm/jaleel/Review_analysis/data/test_null_appended.json


### Visualize tree

In [57]:
import json
from anytree import Node, RenderTree
from typing import List
def build_anytree(node, parent=None):
    """Recursively convert JSON tree into anytree Node."""
    if isinstance(node, str):  # leaf node
        return Node(node, parent=parent)

    current = Node(node["text"], parent=parent)
    for child in node.get("children", []):
        build_anytree(child, current)
    return current

def visualize_tree(tree_path):
    """Load tree JSON and print tree structure."""
    
    with open(tree_path, "r") as f:
        tree = json.load(f)
        
    if(isinstance(tree, List)):
        tree = {"text":"root", "children":tree}

    root = build_anytree(tree)  # tree starts with a dict {"text": "root", ...}

    for pre, fill, node in RenderTree(root):
        print(f"{pre}{node.name}")

# Example usage
visualize_tree(cfg['paths']['category_tree'])


root
├── Structure
│   ├── Topics
│   ├── Organization
│   ├── Order of Topics
│   ├── Difficulty
│   ├── Workload
│   │   ├── Deadlines
│   │   └── null
│   ├── Attendance
│   └── null
├── Evaluation
│   ├── Grading Criteria
│   │   ├── Midsem-Endsem
│   │   ├── Project
│   │   └── null
│   ├── Correctness in Evaluation
│   │   ├── Fairness
│   │   ├── Biasness
│   │   └── null
│   ├── Quality of Questions
│   │   ├── Difficulty
│   │   ├── Relevance with Lecture
│   │   └── null
│   └── null
├── Course Material
│   ├── Lecture Slides
│   │   ├── Quality
│   │   ├── Relevance
│   │   │   ├── Coverage of Topics
│   │   │   └── null
│   │   └── null
│   ├── Labs-Tutorial-Assignment Questions
│   │   ├── Relevance
│   │   ├── Quality
│   │   │   ├── Difficulty
│   │   │   └── null
│   │   └── null
│   ├── Extra Material
│   │   ├── Reference Material
│   │   └── null
│   └── null
├── Lecture Delivery
│   ├── Teaching Assistants (TAs)
│   │   ├── Punctuality
│   │   ├── Pedagogical Skills

In [60]:
visualize_tree(cfg['paths']['data_dir'] + "/miscellaneous/new_category_tree.json")


root
├── Structure
│   ├── Topics
│   ├── Organization
│   ├── Order of Topics
│   ├── Difficulty
│   ├── Workload
│   │   └── Deadlines
│   └── Attendance
├── Evaluation
│   ├── Grading Criteria
│   │   ├── Midsem-Endsem
│   │   └── Project
│   ├── Correctness in Evaluation
│   │   ├── Fairness
│   │   └── Biasness
│   └── Quality of Questions
│       ├── Difficulty
│       └── Relevance with Lecture
├── Course Material
│   ├── Lecture Slides
│   │   ├── Quality
│   │   └── Relevance
│   │       └── Coverage of Topics
│   ├── Labs-Tutorial-Assignment Questions
│   │   ├── Relevance
│   │   └── Quality
│   │       └── Difficulty
│   └── Extra Material
│       └── Reference Material
├── Lecture Delivery
│   ├── Teaching Assistants (TAs)
│   │   ├── Punctuality
│   │   └── Pedagogical Skills
│   │       ├── Effectiveness
│   │       ├── Subject Knowledge
│   │       └── Interactiveness
│   │           └── Approachability
│   └── Professor
│       ├── Punctuality
│       ├── Pace
│       

In [59]:
visualize_tree(cfg['paths']['data_dir'] + "/miscellaneous/category_tree.json")


root
├── Structure
│   ├── Topics
│   │   ├── Relevant
│   │   ├── Irrelevant
│   │   └── Obsolete
│   ├── Organization
│   ├── Order of Topics
│   ├── Difficulty
│   ├── Workload
│   │   └── Deadlines
│   └── Attendance
├── Evaluation
│   ├── Grading Criteria
│   │   ├── Assignment
│   │   ├── Quiz
│   │   ├── Midsem-Endsem
│   │   ├── Project
│   │   ├── Presentation
│   │   ├── Labs-Tutorial-Worksheet
│   │   └── Attendance
│   ├── Correctness in Evaluation
│   │   ├── Fairness
│   │   ├── Biasness
│   │   └── Transparency
│   ├── Solutions
│   │   ├── Timings of Upload
│   │   ├── Correctness
│   │   └── Explanation
│   └── Quality of Questions
│       ├── Correctness & Soundness
│       ├── Difficulty
│       └── Relevance with Lecture
├── Course Material
│   ├── Lecture Slides
│   │   ├── Quality
│   │   │   ├── Correctness
│   │   │   ├── Understandability
│   │   │   ├── Clarity
│   │   │   ├── Organization
│   │   │   └── Presentation
│   │   ├── Timings of Upload
│   │   └── 